## 04 - Understanding the Results

Let's take a look at the outputs generated in part 03. The first thing we need to do is to again load in the libraries we'll want to use.

In [1]:
import sys
sys.path.append('c:\\Users\\tuttl\\OneDrive\\Documents\\My Documents\\KlevitHahn\\hdx-ms\\pyHXExpress\\')

import os
import importlib
import numpy as np, pandas as pd
pd.set_option('display.max_columns',None) 
pd.set_option('display.max_colwidth', None)

import pyhxexpress as hxex
import test_config as config

def hxex_reload():
    importlib.reload(hxex)
    importlib.reload(config)
    hxex.config = config

hxex_reload()

Lets read in the outputs we generated in part 3. These are the metadf_asrun, data_fits, and fitparamsALL files.<p>
We didn't save the raw and peakpicked data to separate csv files, but we can quickly gather those using the 'get_data()' function.

In [6]:
data_path = '..\\Bimodal_HDX_Data\\output_03Mar2024\\'
metadf_run = pd.read_csv(os.path.join(data_path,'metadf_asrun_03Mar2024.csv')).drop('Index',axis=1)
datafits = pd.read_csv(os.path.join(data_path,'data_fits03Mar2024.csv')).drop('Index',axis=1)
fitparams = pd.read_csv(os.path.join(data_path,'fitparamsAll_asrun_03Mar2024.csv')).drop('Index',axis=1)

deutdata, rawdata = hxex.get_data(metadf_run)

In [7]:
# These are the files that were run and the summary of timepoints that are polymodal
display(metadf_run)

,file,sample,start_seq,end_seq,peptide_range,charge,peptide,polymodal,dataset_run
0,Angio_2_HI.xlsx,AngioII_HI,1,8,0001-0008,2,DRVYIHPF,60 120 180 240 300 360 420 480 540 600 660 720 780 840 900 960 1020 1080 1140 1200 1260,Yes
1,GluFib_2_HI.xlsx,GluFib_HI,1,14,0001-0014,2,EGVNDNEEGFFSAR,240 300 360 420 480 540 600 660 720 780 840 900 960 1020 1080 1140 1200 1260,Yes


In [8]:
'''
This is the peak picked data for every peptide/timepoint/charge/replicate corresponding to the 'file's in metadf_run 
There are additional columns which may include 'env_width', 'env_symm', and 'TD_env_width', these will be discussed more
in an advanced topics tutorial. They are 'Features' from the unfit peak picked data that I have used to train an ML model on 
for predicting whether there are 1 or more populations present. 
'''
display(deutdata)

,mz,Intensity,n_deut,env_width,env_symm,max_namides,time,data_id,sample,peptide,charge,rep,peptide_range,start_seq,end_seq,file,time_idx
0,523.774534,574000.0,0,2.574674,2.0,6,0.0,0,AngioII_HI,DRVYIHPF,2,1,0001-0008,1,8,Angio_2_HI.xlsx,0
1,524.277647,352400.0,1,2.574674,2.0,6,0.0,0,AngioII_HI,DRVYIHPF,2,1,0001-0008,1,8,Angio_2_HI.xlsx,0
2,524.780761,103200.0,2,2.574674,2.0,6,0.0,0,AngioII_HI,DRVYIHPF,2,1,0001-0008,1,8,Angio_2_HI.xlsx,0
3,525.283874,21230.0,3,2.574674,2.0,6,0.0,0,AngioII_HI,DRVYIHPF,2,1,0001-0008,1,8,Angio_2_HI.xlsx,0
4,525.786988,4580.0,4,2.574674,2.0,6,0.0,0,AngioII_HI,DRVYIHPF,2,1,0001-0008,1,8,Angio_2_HI.xlsx,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,795.904329,0.0,20,15.143412,1.1,13,1260.0,1,GluFib_HI,EGVNDNEEGFFSAR,2,1,0001-0014,1,14,GluFib_2_HI.xlsx,21
942,796.407442,0.0,21,15.143412,1.1,13,1260.0,1,GluFib_HI,EGVNDNEEGFFSAR,2,1,0001-0014,1,14,GluFib_2_HI.xlsx,21
943,796.910556,0.0,22,15.143412,1.1,13,1260.0,1,GluFib_HI,EGVNDNEEGFFSAR,2,1,0001-0014,1,14,GluFib_2_HI.xlsx,21
944,797.413669,0.0,23,15.143412,1.1,13,1260.0,1,GluFib_HI,EGVNDNEEGFFSAR,2,1,0001-0014,1,14,GluFib_2_HI.xlsx,21


In [9]:
# This is the raw spectral data corresponding to the files in metadf_run
display(rawdata)

,index,mz,Intensity,time,data_id,sample,peptide,charge,rep,peptide_range,start_seq,end_seq,file,time_idx
0,0,523.435,277.00,0.0,0,AngioII_HI,DRVYIHPF,2,1,0001-0008,1,8,Angio_2_HI.xlsx,0
1,1,523.445,236.80,0.0,0,AngioII_HI,DRVYIHPF,2,1,0001-0008,1,8,Angio_2_HI.xlsx,0
2,2,523.455,164.80,0.0,0,AngioII_HI,DRVYIHPF,2,1,0001-0008,1,8,Angio_2_HI.xlsx,0
3,3,523.465,126.00,0.0,0,AngioII_HI,DRVYIHPF,2,1,0001-0008,1,8,Angio_2_HI.xlsx,0
4,4,523.475,107.70,0.0,0,AngioII_HI,DRVYIHPF,2,1,0001-0008,1,8,Angio_2_HI.xlsx,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31954,799,795.230,108.30,1260.0,1,GluFib_HI,EGVNDNEEGFFSAR,2,1,0001-0014,1,14,GluFib_2_HI.xlsx,21
31955,800,795.243,86.75,1260.0,1,GluFib_HI,EGVNDNEEGFFSAR,2,1,0001-0014,1,14,GluFib_2_HI.xlsx,21
31956,801,795.255,62.75,1260.0,1,GluFib_HI,EGVNDNEEGFFSAR,2,1,0001-0014,1,14,GluFib_2_HI.xlsx,21
31957,802,795.267,68.00,1260.0,1,GluFib_HI,EGVNDNEEGFFSAR,2,1,0001-0014,1,14,GluFib_2_HI.xlsx,21


In [10]:
'''
This dataframe contains some of the values we're usually most interested in such as Dabs_ and pop_ and the centroids
Dabs_i is the TD-UN corrected value for the Deuterium uptake with a corresponding population pop_i 

The p-values for any fit_pops > min_pops should be less than the specified 'Ncurve_p_accept' value. The p-value will be 1.0 
for fit_pops = min_pops, meaning additional populations did not sufficiently reduce the p-value. 
'''
display(datafits)

,data_id,sample,peptide,peptide_range,start_seq,end_seq,charge,time,time_idx,rep,centroid,env_width,env_symm,max_namides,fit_pops,p-value,centroid_1,Dabs_1,Dabs_std_1,pop_1,pop_std_1,centroid_2,Dabs_2,Dabs_std_2,pop_2,pop_std_2,centroid_3,Dabs_3,Dabs_std_3,pop_3,pop_std_3,solution_npops
0,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,0,0,1,524.090048,2.574674,2.000000,6,1,1.000000e+00,524.098315,0.000002,0.000008,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,60,1,1,524.917422,5.463998,1.251208,6,2,5.833001e-03,524.758920,1.092169,0.164276,0.563213,0.224458,525.571926,2.401882,0.414734,0.436787,0.224458,NaN,NaN,NaN,NaN,NaN,1
2,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,120,2,1,525.677506,6.611551,1.100000,6,2,8.673665e-03,525.682528,3.117942,0.014304,0.219460,0.155553,525.675457,3.141517,0.003475,0.780540,0.155553,NaN,NaN,NaN,NaN,NaN,1
3,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,180,3,1,526.467321,6.507604,1.100000,6,2,2.905487e-03,526.126745,4.268649,0.238332,0.582173,0.319568,526.581603,5.742099,0.798433,0.417827,0.319568,NaN,NaN,NaN,NaN,NaN,1
4,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,240,4,1,525.444624,8.890903,2.000000,6,2,3.330669e-16,524.114107,0.032626,0.000935,0.542128,0.001256,526.991943,5.738137,0.004878,0.457872,0.001256,NaN,NaN,NaN,NaN,NaN,2
5,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,300,5,1,524.470524,4.665348,2.000000,6,2,3.841372e-14,524.148849,0.100599,0.001562,0.588902,0.001447,524.921184,1.637171,0.004073,0.411098,0.001447,NaN,NaN,NaN,NaN,NaN,2
6,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,360,6,1,524.857859,6.493591,2.000000,6,2,1.110223e-16,524.139262,0.081517,0.000430,0.550295,0.000262,525.720075,3.223113,0.001168,0.449705,0.000262,NaN,NaN,NaN,NaN,NaN,2
7,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,420,7,1,525.169334,7.815740,2.000000,6,2,1.110223e-16,524.113968,0.031021,0.000617,0.536717,0.000232,526.363555,4.497272,0.001193,0.463283,0.000232,NaN,NaN,NaN,NaN,NaN,2
8,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,480,8,1,525.944078,9.399180,1.246850,6,2,1.110223e-16,524.884895,1.564117,0.002052,0.482508,0.000492,526.926104,5.615124,0.001368,0.517492,0.000492,NaN,NaN,NaN,NaN,NaN,2
9,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,540,9,1,525.338626,6.854995,1.157764,6,2,4.359180e-12,524.492704,0.739113,0.005107,0.135237,0.001929,525.471136,2.731339,0.003389,0.864763,0.001929,NaN,NaN,NaN,NaN,NaN,2


In [11]:
'''
The fitparams dataframe contains all of the fit variables determined during the full run. Each fit spectrum will have
fit_pops + Nboot number of entries. Consider the first fit spectrum: the first row is the n=1 fit, the second row is 
the n=2 fit. The n=2 fit did not sufficiently reduce the residual sum squared (rss) value so the Nboot fits are performed
on n=1 populations. 
'''
display(fitparams)

,data_id,sample,peptide,peptide_range,start_seq,end_seq,charge,time,time_idx,rep,ncurves,nboot,rss,Fit_Params,solution_npops,p-value
0,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,0,0,1,1,0,2.400069e-04,0.005248293904662378 0.010412878010109467 1.7706173410742923e-09 0.9999999999,1,1.0
1,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,0,0,1,2,0,2.400069e-04,0.005248293803837907 7.199860626337207 2.252622687499631e-12 8.641904413399275e-15 2.416167927169527e-13 0.7158767676424277 0.9999827482464393,1,1.0
2,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,0,0,1,1,1,1.500147e-05,0.005161130283669983 0.011106913344775956 4.994930828050021e-14 1.0,1,1.0
3,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,0,0,1,1,2,1.500101e-05,0.00525841047126317 0.0019055165240733467 0.0021535688994672917 1.0,1,1.0
4,2,AngioII_HI,DRVYIHPF,0001-0008,1,8,2,0,0,1,1,3,1.500120e-05,0.005322317303004357 0.0002674803448576562 0.000466161346262612 1.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,5,GluFib_HI,EGVNDNEEGFFSAR,0001-0014,1,14,2,1000000,22,1,1,16,9.219232e-07,1.2461714551849375e-11 13.821909219560858 0.8619723125654373 1.0,1,1.0
1047,5,GluFib_HI,EGVNDNEEGFFSAR,0001-0014,1,14,2,1000000,22,1,1,17,9.211911e-07,0.0 13.81957002651802 0.862133435716769 1.0,1,1.0
1048,5,GluFib_HI,EGVNDNEEGFFSAR,0001-0014,1,14,2,1000000,22,1,1,18,9.218768e-07,1.1665469362879447e-12 13.823435233086787 0.8619867307744898 1.0,1,1.0
1049,5,GluFib_HI,EGVNDNEEGFFSAR,0001-0014,1,14,2,1000000,22,1,1,19,9.210390e-07,0.0 13.8195098802929 0.8621477201917084 1.0,1,1.0
